# Загружаем необходимые пакеты и импортируем библиотеки

---

In [1]:
# устанавливаем библиотеку implicit 
%pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 50.2 MB/s eta 0:00:00


In [2]:
# загружаем библиотеки
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
import tensorflow as tf
from sklearn import preprocessing
import ast

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Проверяем что у нас работает GPU
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
#GPU count and name
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-665336b9-dd5d-09c9-0da0-1bcc769d84d8)


In [6]:
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Fri Apr 28 05:11:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W /  70W |    391MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Подготовка и обработка данных

---

In [7]:
# загружаем данные из источника
#data = pd.read_parquet('/content/drive/MyDrive/Hackaton 04.2023/df_train.parquet', engine='pyarrow')
data = pd.read_parquet('/content/drive/MyDrive/Hackaton 04.2023/df_train.parquet', engine='pyarrow')

In [6]:
test = pd.read_csv('/content/drive/MyDrive/Hackaton 04.2023/df_test_list_top.csv')
test.columns=[['cookie_id','vacancy_id_']]
test


,cookie_id,vacancy_id_
0,00252da2760d4231b366d79eaf6a54a1,"[139502, 133937, 153196, 166055, 168696]"
1,0052234b5356480f9dff207888135548,"[108861, 134982, 141785, 146245, 153937]"
2,007090a8c8354b42bf538b93785303d3,"[101396, 118250, 122614, 152485, 164067]"
3,00c15cb6e3c84464a0e538e46ced6c46,"[139462, 101462, 108276, 108536, 111783]"
4,00d6baa0391e41628da60e2b4a37e19f,"[117958, 157903, 159843, 164325, 164331]"
...,...,...
1785,ff6aeeddb38847c59979efce29225e87,"[112920, 126057, 126404, 127524, 130124]"
1786,ff7861c402e84f9fa78ff00f49eb83a1,"[248949, 254085, 108510, 128900, 182528]"
1787,ff81d0fe535247ac9df2468f8e57a9e9,"[107949, 104587, 134588, 136000, 143178]"
1788,ffa618b6b94444239aee6356fc67af48,"[101136, 117182, 117525, 185355, 223343]"


In [8]:
def get_dict_from_columns(col1, col2):
    dict_from_cols = {}
    for i in range(len(col2)):
        if col2[i] not in dict_from_cols:
            dict_from_cols[col2[i]] = col1[i]
    return dict_from_cols
user_dict = get_dict_from_columns(data['cookie_id'],data['user_id'])

In [9]:
data['new_cookie'] = data['user_id'].map(user_dict)

In [10]:
data['key'] = data['new_cookie'].astype(str)+'_'+data['vacancy_id_num'].astype(str)+'_'+data['Rating'].astype(str)
data.head(5)

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,Rating,vacancy_id_num,cookie_id_num,user_id_num,key,new_cookie
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy,0,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,97990f1a021d4be19aa3f955b7eacab4_129850_0,97990f1a021d4be19aa3f955b7eacab4
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_108347_0,03bf8c511fa949c79845a5d81b09aa1d
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_109069_0,03bf8c511fa949c79845a5d81b09aa1d
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_171425_0,03bf8c511fa949c79845a5d81b09aa1d
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_252384_0,03bf8c511fa949c79845a5d81b09aa1d


In [10]:
def filter_by_unique_value(df, col_name):
    # Get unique values based on the specified column
    unique_values = df[col_name].unique()

    # Filter the original DataFrame based on the unique values
    filtered_df = df[df[col_name].isin(unique_values)]

    # Return the filtered DataFrame with all the original columns
    return filtered_df


In [11]:
data_train = filter_by_unique_value(data,'key')
data_train['cookie_id']=data_train['new_cookie']

In [12]:
# создаем копию
df = data_train.copy()

In [14]:
data['cookie_id'] = data['new_cookie']
df = data.copy()

In [15]:
153196 in set(df[df['cookie_id']=='00252da2760d4231b366d79eaf6a54a1']['vacancy_id_num'])

False

In [16]:
# посмотрим на данные 
df.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,Rating,vacancy_id_num,cookie_id_num,user_id_num,key,new_cookie
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy,0,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,97990f1a021d4be19aa3f955b7eacab4_129850_0,97990f1a021d4be19aa3f955b7eacab4
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_108347_0,03bf8c511fa949c79845a5d81b09aa1d
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_109069_0,03bf8c511fa949c79845a5d81b09aa1d
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_171425_0,03bf8c511fa949c79845a5d81b09aa1d
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,03bf8c511fa949c79845a5d81b09aa1d_252384_0,03bf8c511fa949c79845a5d81b09aa1d


In [15]:
# и так тоже посмотрим
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12234390 entries, 0 to 12234389
Data columns (total 12 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   event_date       object
 1   event_timestamp  int64 
 2   vacancy_id_      int64 
 3   cookie_id        object
 4   user_id          object
 5   event_type       object
 6   Rating           int64 
 7   vacancy_id_num   int64 
 8   cookie_id_num    object
 9   user_id_num      object
 10  key              object
 11  new_cookie       object
dtypes: int64(4), object(8)
memory usage: 1.2+ GB


In [17]:
# удаляем колонки, которые не будут принимать участия в обучении
df.drop(columns=['event_date', 'event_timestamp', 'user_id'], axis=1, inplace=True)

In [18]:
# для удобства немного выравняем данные
df = df[['cookie_id', 'vacancy_id_', 'event_type']]

In [19]:
df.head()

,cookie_id,vacancy_id_,event_type
0,97990f1a021d4be19aa3f955b7eacab4,129850,show_vacancy
1,03bf8c511fa949c79845a5d81b09aa1d,108347,show_vacancy
2,03bf8c511fa949c79845a5d81b09aa1d,109069,show_vacancy
3,03bf8c511fa949c79845a5d81b09aa1d,171425,show_vacancy
4,03bf8c511fa949c79845a5d81b09aa1d,252384,show_vacancy


 ### Обработка данных первая исследовательская позиция (Маштабирование взаимодействий и их силы)
 ---

In [39]:
# Вот тут мы посмотрим на количество определенного вида взаимодействий в данных
df['event_type'].value_counts()

show_vacancy              6164618
preview_click_vacancy     4745522
click_response             381819
click_contacts             268029
preview_click_response     189382
click_favorite             155081
preview_click_favorite     106279
preview_click_contacts      92545
click_phone                 73336
preview_click_phone         12475
Name: event_type, dtype: int64

* 1 show_vacancy - просмотр вакансии
* 2 preview_click_vacancy - клик по карточке вакансии
* **3 click_response - отклик со страницы вакансии**
* **4 preview_click_response - отклик с карточки вакансии**
* 5 click_favorite - добавление вакансии в избранное  со страницы вакансии
* 6 preview_click_favorite - добавление вакансии в избраное с карточки вакансии
* **7 click_contacts - клик на контакты со страницы вакансии**
* **8 preview_click_contacts - клик на контакты из карточки вакансии**
* **9 click_phone - клик на номер телефона, указанный в вакансии**
* **10 preview_click_phone - клик на номер телефона из карточки вакансии**

Жирным выделены искомые ""откликнется"" и ""позвонит"""
! Стоит обратить внимание на то, что позиции 3 и 4 по частоте идут выше чем позиции 5 и 6, которые нам в положительное взаимодействие не засчитываются. В свзи с этим необходимо проработать несколько вариантов маштабирования данных

In [20]:
# проверим если закодировать значимость взаимодейстивий по их количеству в выборке - при прочих стандартных параметрах этот
# способ показал результат 0,041
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 2.0,
                                     'click_response': 3.0,
                                     'click_contacts': 4.0,
                                     'preview_click_response': 5.0,
                                     'click_favorite': 6.0,
                                     'preview_click_favorite': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [41]:
# проверим правильно ли мы закодировали взаимодействия
df['eventStrength'].value_counts()

1.0     6164618
2.0     4745522
3.0      381819
4.0      268029
5.0      189382
6.0      155081
7.0      106279
8.0       92545
9.0       73336
10.0      12475
Name: eventStrength, dtype: int64

Далее несколько ячеек кода для различных вариаций оценки взаимодействия их стоит прогнать с базовыми значениями и посмотреть на результаты (возможны и другие предложения):

In [ ]:
# проверить потом - вот такая модель должна дать нам больший перекос в сторону искомых взаимодействий перед обычными
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 1.0,
                                     'click_favorite': 1.0,
                                     'preview_click_favorite': 1.0,
                                     'click_response': 10.0,
                                     'preview_click_response': 10.0,
                                     'click_contacts': 10.0,
                                     'preview_click_contacts': 10.0,
                                     'click_phone': 10.0,
                                     'preview_click_phone': 10.0})

In [ ]:
# Вот это разбиение по нашему заданию, то есть первые 4 отрицательные взаимодействия остальные положительные
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 2.0,
                                     'click_favorite': 3.0,
                                     'preview_click_favorite': 4.0,
                                     'click_response': 5.0,
                                     'click_contacts': 6.0,
                                     'preview_click_response': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [ ]:
# Вот это разбиение по нашему заданию, но незначимые мы закодируем нулями нужные единицами, но не знаю как это пойдет
# Нули в матрице могут восприниматься как отсутсвие взаимодействия, а это неправда
df['eventStrength'] = df['event_type'].map({'show_vacancy': 0.0,
                                     'preview_click_vacancy': 0.0,
                                     'click_favorite': 0.0,
                                     'preview_click_favorite': 0.0,
                                     'click_response': 1.0,
                                     'click_contacts': 1.0,
                                     'preview_click_response': 1.0,
                                     'preview_click_contacts': 1.0,
                                     'click_phone': 1.0,
                                     'preview_click_phone': 1.0})

In [ ]:
# Вот это разбиение по нашему заданию, но незначимые мы закодируем 1 нужные 2
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 1.0,
                                     'click_favorite': 1.0,
                                     'preview_click_favorite': 1.0,
                                     'click_response': 2.0,
                                     'click_contacts': 2.0,
                                     'preview_click_response': 2.0,
                                     'preview_click_contacts': 2.0,
                                     'click_phone': 2.0,
                                     'preview_click_phone': 2.0})

In [ ]:
# Вот это разбиение по нашему заданию, ну и последний вариант это вот так:
df['eventStrength'] = df['event_type'].map({'show_vacancy': 0.1,
                                     'preview_click_vacancy': 0.1,
                                     'click_favorite': 0.1,
                                     'preview_click_favorite': 0.1,
                                     'click_response': 1.0,
                                     'click_contacts': 1.0,
                                     'preview_click_response': 1.0,
                                     'preview_click_contacts': 1.0,
                                     'click_phone': 1.0,
                                     'preview_click_phone': 1.0})

Группировка данных:

---

In [21]:
# сгруппируем сочетания ваканции и человека при этом суммируем историю из взаимодействия в числах, чем больще число
# тем больше интерес человека к вакансии - а значит больше и сила взаимодействия

grouped_df = df.groupby(['cookie_id', 'vacancy_id_']).sum('numeric_only').reset_index()

In [43]:
# с 12 000 000 + наши данные схлопнулись до 4 500 000 +
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4669635 entries, 0 to 4669634
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   cookie_id      object 
 1   vacancy_id_    int64  
 2   eventStrength  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 106.9+ MB


In [44]:
# посмотрим как они выглядят теперь, далее можно будет сравнить как выглядит одно и тоже сравнение в контексте
# других вариантов маштабирования силы взаимодействий 
grouped_df.sample(10)

,cookie_id,vacancy_id_,eventStrength
2277046,7c39c78dfabc48448f0870ff0ece7789,254953,3.0
4147850,e317232cc8b745f39610edb8e477e0e8,211848,3.0
2787858,97e22f7923cd48a294b5dc0cdaec754f,198114,3.0
405818,163fc7109a1740768257de20f294ce39,258441,3.0
786190,2ad9c9a1fad9434dabdd67ce968989a6,216845,3.0
2891362,9d924ea20dcf4833bec1418c27ae462e,107949,11.0
3196979,aebb8f1407234a3391e75e6b9ce4d4a3,258114,25.0
1309690,478d94b0ba8d4f9cb48347e69e1310e7,208788,3.0
379347,14d1af21b11244d08fd854ec685610ef,200403,3.0
2726829,948a8cec3fad4300b2126a6c5dcbc1be,141610,3.0


In [21]:
# посмотрим на одну вакансию в ключе ее взаимодействия с пользователями
grouped_df[grouped_df['vacancy_id_'] == 210068].sort_values('eventStrength', ascending=False).head(10)

,cookie_id,vacancy_id_,eventStrength
869322,3c99e2dfbc75461fa99f9b81c1000735,210068,103.0
395560,1e9223caa87c453da3a102f3020327f7,210068,27.0
2344618,ac5710432b184069b1a4c0296fe3e41a,210068,12.0
3348528,fb3eebb8cfc54a3085a07e7f47dbed4c,210068,10.0
220791,110a687ec500431289aa6116974ffc36,210068,9.0
1024863,45d92c30209a4ef0bf45166ca2a9d9ae,210068,9.0
529116,2901a05b78a0471a9f5ad505ee70168d,210068,9.0
999274,43de234dae2a41f98f394d8e1e615528,210068,9.0
1464589,68165fd867784624880084ae3a8de2a5,210068,7.0
2367617,ae48689ab98c4fc6acc9b848ac03a54c,210068,6.0


In [22]:
# посмотрим на одного пользователя в ключе его взаимодействия с вакансиями
grouped_df[grouped_df['cookie_id'] == 'ac5710432b184069b1a4c0296fe3e41a'].sort_values('eventStrength', ascending=False).head(10)

,cookie_id,vacancy_id_,eventStrength
2344610,ac5710432b184069b1a4c0296fe3e41a,168822,23.0
2344606,ac5710432b184069b1a4c0296fe3e41a,136830,21.0
2344617,ac5710432b184069b1a4c0296fe3e41a,195789,20.0
2344623,ac5710432b184069b1a4c0296fe3e41a,239208,13.0
2344618,ac5710432b184069b1a4c0296fe3e41a,210068,12.0
2344605,ac5710432b184069b1a4c0296fe3e41a,131067,11.0
2344620,ac5710432b184069b1a4c0296fe3e41a,215541,10.0
2344619,ac5710432b184069b1a4c0296fe3e41a,214215,10.0
2344614,ac5710432b184069b1a4c0296fe3e41a,183452,9.0
2344622,ac5710432b184069b1a4c0296fe3e41a,221544,9.0


Вот к этому блоку нужно подойти очень внимательно - это выделение категориальных и числовых столбцов, в первых прогонах мы обошлись вот таким вариантом, но можно будет посмотреть и выделение нумированных вакансий (у нас в матрице с 0 до 100 000 значений нет, а номера есть, но может этого и ничего страшного)

---

In [22]:
grouped_df['cookie_id'] = grouped_df['cookie_id'].astype("category") # наименование вакансии в категорию
#grouped_df['vacancy_id_'] = grouped_df['vacancy_id_'].astype("category") # наименование вакансии в категорию

In [23]:
grouped_df['cookie_id_num'] = grouped_df['cookie_id'].cat.codes # вот тут выделяем числовые значения
#grouped_df['vacancy_id_num'] = grouped_df['vacancy_id_'].cat.codes # и вот тут выделим

Продолжаем готовить данные для разряженных матриц

---

In [ ]:
#grouped_df = grouped_df[['cookie_id', 'cookie_id_num', 'vacancy_id_', 'vacancy_id_num', 'eventStrength']]

In [24]:
grouped_df = grouped_df[['cookie_id', 'cookie_id_num', 'vacancy_id_', 'eventStrength']]
#grouped_df = grouped_df[['cookie_id_num', 'vacancy_id_num', 'eventStrength']]

In [25]:
data = grouped_df[['cookie_id_num', 'vacancy_id_', 'eventStrength']]

In [26]:
# В этом варианте кода мы пробуем запускать в матрицу номера вакансий которые нам дали изначально, что бы с ними потом не мучатся
data = grouped_df[['cookie_id_num', 'vacancy_id_', 'eventStrength']]

Вот тут стоит подумать о нормализации данных и вообще нужна ли она? Это второй исследовательский блок давайте пробовавать разную нормализацию

In [ ]:
data['eventStrength_norm'] = (data['eventStrength'] - data['eventStrength'].mean()) / data['eventStrength'].std()

In [ ]:
data.head() # мне кажется минусы нам не нужны будут проверю другую

In [ ]:
data['eventStrength_norm'] = (data['eventStrength'] - data['eventStrength'].min()) / (data['eventStrength'].max() - data['eventStrength'].min())

In [55]:
data.head() # мне кажется минусы нам не нужны будут проверю другую
# Вот этот датафрейм пойдет в матрицы (в следующий прогон загоним с нормализацией)

,cookie_id_num,vacancy_id_,eventStrength
0,0,137659,3.0
1,0,153975,3.0
2,0,174953,9.0
3,0,176171,3.0
4,0,182445,6.0


In [57]:
# а вот это остается нашим базовым информационным датафреймом
grouped_df.head(10)

,cookie_id,cookie_id_num,vacancy_id_,eventStrength
0,0000c4548c3944c08972bbdc1fa4eb85,0,137659,3.0
1,0000c4548c3944c08972bbdc1fa4eb85,0,153975,3.0
2,0000c4548c3944c08972bbdc1fa4eb85,0,174953,9.0
3,0000c4548c3944c08972bbdc1fa4eb85,0,176171,3.0
4,0000c4548c3944c08972bbdc1fa4eb85,0,182445,6.0
5,0000c4548c3944c08972bbdc1fa4eb85,0,187529,3.0
6,0000e63f423542fe8090575b611cea4d,1,122287,13.0
7,0000e63f423542fe8090575b611cea4d,1,131966,6.0
8,0000e63f423542fe8090575b611cea4d,1,137592,6.0
9,0000e63f423542fe8090575b611cea4d,1,141592,7.0


Вот в этом блоке мы должны соотнести наш цифровой номер cookie со строчным, для вывода ответов

---

Первый вариант работает на словарях

In [27]:
# кодируем числами все наши cookie_id
codes = grouped_df.cookie_id.astype('category')
# создаем словарик в котором ключ это код а значение текстовое поле cookie_id
cookie_dic = dict(enumerate(codes.cat.categories))
print (codes.cat.categories)

Index(['0000c4548c3944c08972bbdc1fa4eb85', '0000e63f423542fe8090575b611cea4d',
       '00015019b0594984afc3fd9fa9557490', '0001a23eb7fa4b1db1e933484b03e668',
       '00028e0b0d474748ac5a83688a4f8896', '0002cb657088464b99775a69ce83a2ef',
       '000344ddc0664127b54f529eaf183e30', '000385da780542d5a2437cc416e4f063',
       '00038b5200304d5d96a05a31ca94751a', '0003e60e0c684618baf363fcc19bc2a4',
       ...
       'fffd7799244e4e3d82ea3ed22268f67a', 'fffd83f9c6fb4d7fb899e133af17227c',
       'fffdb1e5a85841eba7bce22e11e1e0bd', 'fffdc951d3b64eb5b1de9195f3df9920',
       'fffe6d5c21f4400d97c826f2e61e9222', 'fffeba46c96a47f4a2ff364500757de3',
       'fffeca6b3b8842d1b7582956e660d463', 'ffff3581b7e7405db25fac5c5dd1bff9',
       'ffff8283d29644e69ad6d08cc3ca7fb7', 'ffffd6d4f2134dd6aa80ff71e942b508'],
      dtype='object', length=208238)


In [28]:
# и создадим обратный словарик
cookie_to_code = dict(zip(cookie_dic.values(), cookie_dic.keys()))

In [29]:
grouped_df

,cookie_id,cookie_id_num,vacancy_id_,eventStrength
0,0000c4548c3944c08972bbdc1fa4eb85,0,137659,3.0
1,0000c4548c3944c08972bbdc1fa4eb85,0,153975,3.0
2,0000c4548c3944c08972bbdc1fa4eb85,0,174953,9.0
3,0000c4548c3944c08972bbdc1fa4eb85,0,176171,3.0
4,0000c4548c3944c08972bbdc1fa4eb85,0,182445,6.0
...,...,...,...,...
3409002,ffffd6d4f2134dd6aa80ff71e942b508,208237,171308,3.0
3409003,ffffd6d4f2134dd6aa80ff71e942b508,208237,197828,7.0
3409004,ffffd6d4f2134dd6aa80ff71e942b508,208237,225522,9.0
3409005,ffffd6d4f2134dd6aa80ff71e942b508,208237,230870,4.0


In [30]:
print(cookie_to_code['ffffd6d4f2134dd6aa80ff71e942b508'])

208237


Другой способ через dataFrame пока заморозим

In [31]:
cookie_dic_df = grouped_df.drop(columns = ['vacancy_id_','eventStrength'])

In [32]:
cookie_dic_df.drop_duplicates(inplace=True)

In [ ]:
# вот у нас и справочник потом будем его мержить
cookie_dic_df.head()

,cookie_id,cookie_id_num
0,0000c4548c3944c08972bbdc1fa4eb85,0
6,0000d7508334414ca792c5ff66eb8c14,1
18,0000e63f423542fe8090575b611cea4d,2
26,00013bf3e58244b48d4dbdf43543c30c,3
32,00015019b0594984afc3fd9fa9557490,4


In [33]:
# проверим наш предыдущий метод и попробуем на cookie_dic_df крутить туда и сюда код в номер и обратно
cookie_dic_df['cookie_id_back'] = cookie_dic_df['cookie_id_num'].map(cookie_dic)

In [34]:
cookie_dic_df['cookie_id_to_code'] = cookie_dic_df['cookie_id'].map(cookie_to_code)

In [35]:
# проверка пройдена
cookie_dic_df.head()

,cookie_id,cookie_id_num,cookie_id_back,cookie_id_to_code
0,0000c4548c3944c08972bbdc1fa4eb85,0,0000c4548c3944c08972bbdc1fa4eb85,0
6,0000e63f423542fe8090575b611cea4d,1,0000e63f423542fe8090575b611cea4d,1
14,00015019b0594984afc3fd9fa9557490,2,00015019b0594984afc3fd9fa9557490,2
36,0001a23eb7fa4b1db1e933484b03e668,3,0001a23eb7fa4b1db1e933484b03e668,3
81,00028e0b0d474748ac5a83688a4f8896,4,00028e0b0d474748ac5a83688a4f8896,4


In [ ]:
data['cookie_id_num'].head(5)
data


,cookie_id_num,vacancy_id_,eventStrength
0,0,137659,3.0
1,0,153975,3.0
2,0,174953,9.0
3,0,176171,3.0
4,0,182445,6.0
...,...,...,...
4669630,330179,107125,7.0
4669631,330179,107893,10.0
4669632,330179,150843,10.0
4669633,330179,160164,10.0


Создание разряженных матриц

---

In [ ]:
# вот это уже проверенный код но он работает на кодированных номерах и cookie и vacancy_id - мы его отставили в сторону

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['vacancy_id_num'], grouped_df['cookie_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['cookie_id_num'], grouped_df['vacancy_id_num'])))

In [36]:
# а вот эта ячейка экспериментальный, главный ее смысл понять что базовые данные vacancy_id_ работают, мы провели эксперимент
# и они работают сейчас это базовые матрицы

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((data['eventStrength'].astype(float), (data['vacancy_id_'], data['cookie_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((data['eventStrength'].astype(float), (data['cookie_id_num'], data['vacancy_id_'])))

In [ ]:
# предыдущий код но с нормализацией

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((data['eventStrength_norm'].astype(float), (data['vacancy_id_'], data['cookie_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((data['eventStrength_norm'].astype(float), (data['cookie_id_num'], data['vacancy_id_'])))

In [37]:
# проверим размерности
sparse_item_user.shape
# стоит обратить внимание на то что размерность вакансий 260 168 - это потому что первых 100000 вакансий нет
# если бы мы кодировали - то размерность бы была 160 000 что то такое

(260168, 208238)

In [38]:
# и тут проверим
sparse_user_item.shape

(208238, 260168)

In [39]:
# ну и посмотрим что там внутри
print(sparse_user_item[0])

  (0, 137659)	3.0
  (0, 153975)	3.0
  (0, 174953)	9.0
  (0, 176171)	3.0
  (0, 182445)	6.0
  (0, 187529)	3.0


In [40]:
# и тут тоже посмотрим
print(sparse_item_user[137659])

  (0, 0)	3.0
  (0, 86)	3.0
  (0, 164)	2.0
  (0, 319)	1.0
  (0, 376)	3.0
  (0, 428)	6.0
  (0, 506)	3.0
  (0, 708)	1.0
  (0, 836)	3.0
  (0, 849)	3.0
  (0, 1061)	1.0
  (0, 1406)	3.0
  (0, 1412)	3.0
  (0, 1548)	3.0
  (0, 1728)	3.0
  (0, 1895)	6.0
  (0, 2054)	3.0
  (0, 2199)	3.0
  (0, 2483)	2.0
  (0, 2711)	3.0
  (0, 3060)	3.0
  (0, 3124)	4.0
  (0, 3191)	3.0
  (0, 3374)	1.0
  (0, 3469)	3.0
  :	:
  (0, 205146)	3.0
  (0, 205295)	1.0
  (0, 205316)	3.0
  (0, 205511)	3.0
  (0, 205616)	1.0
  (0, 205708)	3.0
  (0, 205728)	1.0
  (0, 205743)	3.0
  (0, 205938)	3.0
  (0, 206111)	3.0
  (0, 206241)	3.0
  (0, 206689)	3.0
  (0, 206825)	3.0
  (0, 206893)	3.0
  (0, 206895)	3.0
  (0, 206999)	3.0
  (0, 207005)	3.0
  (0, 207133)	3.0
  (0, 207359)	3.0
  (0, 207397)	14.0
  (0, 207431)	3.0
  (0, 207498)	17.0
  (0, 207528)	3.0
  (0, 208024)	3.0
  (0, 208232)	10.0


# Обучение моделей
Вот это третий исследовательский этап, мы должны много! очень много экспериментировать с:
 - factors = ?,
 - regularization = ?,
 - iterations = ?

---

In [41]:
# Инициируем наши модели 
model_item = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

In [42]:
model_user = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

Четвертый исследовательский блок связан с параметром альфа - он закомментирован в коде ниже. В тех статьях что я успел прочитать это чуть ли не главнй парметр, непонятно почему конечно, возможно он увеличивает и акцентирует отличие силе взаимодействия?, но тогда стоит задуматься нужна ли нам вообще нормализация? (в статьях встречались значения альфа в 15 и 50, надо пробовать)

Обучаем первую модель вакансия-пользователь (но мы ее скорее всего не будем использовать)

In [43]:
# Вот эти параметры я подсмотрел, они не обязательные но наверное их лучше потом настраивать
#alpha = 50
#item_user_data = (sparse_item_user * alpha).astype('double')

item_user_data = sparse_item_user
# обучим модель на разряженной матрице  item/user/confidence weights - по русски вакансия/пользователь/веса заинтересованности
model_item.fit(item_user_data)

  0%|          | 0/50 [00:00<?, ?it/s]

<260168x330180 sparse matrix of type '<class 'numpy.float64'>'
	with 4669635 stored elements in Compressed Sparse Row format>

Обучаем первую модель пользователь - вакансия (вот ее в основном и принимают наши функции которые мы будем использовать)

In [44]:
# Вот эти параметры я подсмотрел, они не обязательные но наверное их лучше потом настраивать
#alpha = 50
#user_item_data = (sparse_user_item * alpha).astype('double')

user_item_data = sparse_user_item
# train the model on a sparse matrix of user/item confidence weights
model_user.fit(user_item_data)

  0%|          | 0/50 [00:00<?, ?it/s]

# Пробуем извечь пользу. То есть переходим к предсказаниям

---
(ниже собраны базовые функции библиотеки implicit)

## Создаем рекомендации для человека

---

Функция : recommend( )

Рекомендует элементы для пользователя
Вычисляет N лучших рекомендаций для пользователя и возвращает список идентификаторов элементов, оценка.

Параметры:

- userid ( int ) — идентификатор пользователя для расчета рекомендаций для
- user_items ( csr_matrix ) — разреженная матрица формы (number_users, number_items). Это позволяет нам искать понравившиеся предметы и их веса для пользователя. Это используется для фильтрации элементов, которые уже понравились, из выходных данных, а также для потенциального вычисления лучших элементов для этого пользователя.
- N ( int , необязательный ) — количество возвращаемых результатов
- filter_already_liked_items ( bool , необязательный ) — если установлено значение true, не возвращать элементы, присутствующие в обучающем наборе, которые были оценены указанным пользователем.
- filter_items ( последовательность целых чисел , необязательный ) — список дополнительных идентификаторов элементов, которые нужно отфильтровать из вывода.
- recalculate_user ( bool , необязательный ) — когда true, не полагайтесь на сохраненное состояние пользователя и вместо этого пересчитывайте из переданных в user_items

Возвращает:	
Список кортежей (itemid, score)

Тип возврата:	
список

Получим список рекомендаций для одного пользователя

---

In [45]:
# модель тренируется на sparse matrix of user/item/confidence weights - выполнено
# model.fit(user_item_data) - выполнено

# порекомендуем вакансии для одного пользователя из тех что он еще не видел
userid = 0
recommendations = model_user.recommend(userid, user_item_data[userid], N = 5 , filter_already_liked_items = True)
print(recommendations) # уже отсортировано

(array([116823, 182870, 169194, 207423, 158242], dtype=int32), array([0.43271327, 0.43194753, 0.3787542 , 0.30962843, 0.3051292 ],
      dtype=float32))


In [46]:
# или так мы можем сразу превращать их в список
userid = 0
ids, scores = model_user.recommend(userid, user_item_data[userid], N=5, filter_already_liked_items=True)
print(list(ids))
print(scores)

[116823, 182870, 169194, 207423, 158242]
[0.43271327 0.43194753 0.3787542  0.30962843 0.3051292 ]


In [ ]:
# порекомендуем вакансии для одного пользователя из тех что он уже взаимодействовал
userid = 0
recommendations = model_user.recommend(userid, user_item_data[userid], N = 5, filter_already_liked_items = False)
print(recommendations) # уже отсортировано

(array([174953, 116823, 182870, 169194, 162187], dtype=int32), array([0.70582664, 0.44180346, 0.4205275 , 0.37820446, 0.31708193],
      dtype=float32))


Получим списко рекомендаций для списка пользователей

---

In [47]:
# Такой код позволяет получать рекомендации для списка пользователей сразу!!!
userids = np.arange(1000) # вот тут у нас список пользователей
ids, scores = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)
ids, ids.shape

(array([[116823, 182870, 169194, 207423, 158242],
        [111505, 148714, 207423, 237341, 257631],
        [113707, 153245, 148714, 258441, 116823],
        ...,
        [250327, 235614, 117897, 101396, 164236],
        [230322, 230326, 200506, 221419, 195702],
        [202608, 195702, 221419, 207108, 181745]], dtype=int32),
 (1000, 5))

In [48]:
userids = np.arange(1000) # вот тут у нас список пользователей
group_recommendations = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)


In [49]:
# Вот тут нужно написать код который записывает данные в датафрейм
result = pd.DataFrame({'пользователи' : userids, 'рекомендации' : list(ids)})

In [ ]:
result.head()

,пользователи,рекомендации
0,0,"[116823, 182870, 169194, 162187, 180382]"
1,1,"[217683, 260154, 163708, 210229, 115589]"
2,2,"[148714, 112506, 258441, 110792, 113707]"
3,3,"[242145, 153469, 126489, 197991, 207356]"
4,4,"[113707, 153245, 148714, 149024, 116823]"


## Найдем похожие вакансии на вакансию №

---

Функция: similar_items()

Вычисляет список похожих элементов

Параметры:	
- itemid (int) – идентификатор строки элемента, для которого извлекаются похожие элементы
- N (int, необязательно) – количество похожих элементов для возврата.
- react_users (csr_matrix, необязательно) – разреженная матрица формы (number_items, number_users). Это позволяет нам просмотреть откликнувшихся пользователей и их веса для элемента.
- recallculate_item (bool, необязательно) – При значении true не полагайтесь на сохраненное состояние элемента и вместо этого пересчитывайте из переданного в react_users

ВОЗВРАТ:	
Список кортежей (itemid, оценка)

Возвращаемый тип:	
Список

In [ ]:
# Найти похожую вакансию на вакансию №
itemid = 100005
related = model_user.similar_items(itemid)
print(list(related[0])) # стоит обратить внимание на то, что первой выходит сама эта вакансия
print(list(related[0][1:]))

[100005, 188746, 214051, 221652, 200167, 200319, 231082, 225737, 198021, 238321]
[188746, 214051, 221652, 200167, 200319, 231082, 225737, 198021, 238321]


## Найдем похожего пользователя

Функция: similar_users( )

Вычисляет список похожих пользователей

Параметры:

- userid ( int ) — идентификатор строки пользователя для получения похожих пользователей для
- N ( int , необязательный ) — количество похожих пользователей, которых необходимо вернуть.

Возвращает:	
Список кортежей (userid, score)

Тип возврата:	
список

In [ ]:
userid = 100005
related_users = model_user.similar_users(userid)
print(list(related_users[0])) # стоит обратить внимание на то, что первой выходит этот самый пользователь
print(list(related_users[0][1:]))

[100005, 113260, 281362, 58263, 131196, 6227, 143338, 66705, 140260, 307425]
[113260, 281362, 58263, 131196, 6227, 143338, 66705, 140260, 307425]


## Ранжируем элементы для пользователя

---

Функция: rank_items()

Ранжирует заданные элементы для пользователя и возвращает отсортированный список элементов.

Параметры:

- userid (int) – идентификатор пользователя для вычисления рекомендаций
- user_items (csr_matrix) – разреженная матрица формы (number_users, number_items). Это позволяет нам (необязательно) пересчитывать пользовательские факторы (см. параметр reconculate_user) по мере необходимости
- selected_items (List of itemids) – (список идентификаторов элементов)
- recalculate_user (bool, необязательно) – При значении true не полагайтесь на сохраненное состояние пользователя и вместо этого пересчитывайте из переданных в user_items



Возвращаемый тип:	
Список

In [ ]:
userid = 100005
rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004])
print(rank_elements) 


(array([100002, 100001, 100003, 100004]), array([1.981631e-03, 5.119864e-04, 5.209075e-05, 3.583861e-05],
      dtype=float32))


<ipython-input-59-0aa5fe56e486>:2: DeprecationWarning: rank_items is deprecated. Use recommend with the 'items' parameter instead
  rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004])


# Переходим к испытаниям на нашей ситуации

---

Загружаем данные

In [39]:
# Вот это данные от партнера, стоит отметить что там есть строки с 2 вакансиями и более 5
test = pd.read_parquet('/content/drive/MyDrive/Hackaton 04.2023/test_public_mfti.parquet', engine='pyarrow')

In [ ]:
type(test['cookie_id'])

pandas.core.series.Series

In [50]:
test = pd.read_csv('/content/drive/MyDrive/Hackaton 04.2023/df_test_list_top.csv')
test.columns=[['cookie_id','vacancy_id_']]


In [73]:
test.head()


,cookie_id,vacancy_id_
0,00252da2760d4231b366d79eaf6a54a1,"[139502, 133937, 153196, 166055, 168696]"
1,0052234b5356480f9dff207888135548,"[108861, 134982, 141785, 146245, 153937]"
2,007090a8c8354b42bf538b93785303d3,"[101396, 118250, 122614, 152485, 164067]"
3,00c15cb6e3c84464a0e538e46ced6c46,"[139462, 101462, 108276, 108536, 111783]"
4,00d6baa0391e41628da60e2b4a37e19f,"[117958, 157903, 159843, 164325, 164331]"


In [51]:
# получаем для них столбец с номерными вариантами cookie как мы это закодировали ранее
test['cookie_id_num'] = test['cookie_id'].squeeze().map(cookie_to_code)

#test['cookie_id_num'] = test['cookie_id'].apply(cookie_to_code)


In [52]:
test = test.dropna()

In [53]:
test.head()

,cookie_id,vacancy_id_,cookie_id_num
0,00252da2760d4231b366d79eaf6a54a1,"[139502, 133937, 153196, 166055, 168696]",115.0
2,007090a8c8354b42bf538b93785303d3,"[101396, 118250, 122614, 152485, 164067]",350.0
3,00c15cb6e3c84464a0e538e46ced6c46,"[139462, 101462, 108276, 108536, 111783]",611.0
5,00e61f4a8a9f4b8c9fe2913bca7df8d2,"[105265, 109573, 109891, 123183, 182957]",735.0
6,00fa030c79ec4fe5a7b806e6ba60d5ae,"[167159, 211100, 127800, 179478, 189398]",798.0


In [54]:
cookie_to_code['00fa030c79ec4fe5a7b806e6ba60d5ae']

798

In [88]:
# проверим перевод в коды на правильность
print(cookie_to_code['007090a8c8354b42bf538b93785303d3'])
print(cookie_to_code['00c15cb6e3c84464a0e538e46ced6c46'])
#print(cookie_to_code['00d6baa0391e41628da60e2b4a37e19f'])


350
611


In [55]:
# получаем из датафрейма список пользователей, которых мы загрузим в модель
test_list = list(test['cookie_id_num'].squeeze())
test_list_str = list(test['cookie_id'].squeeze())
print(test_list[:5])
print(test_list_str[:5])

[115.0, 350.0, 611.0, 735.0, 798.0]
['00252da2760d4231b366d79eaf6a54a1', '007090a8c8354b42bf538b93785303d3', '00c15cb6e3c84464a0e538e46ced6c46', '00e61f4a8a9f4b8c9fe2913bca7df8d2', '00fa030c79ec4fe5a7b806e6ba60d5ae']


In [56]:
# Ячейка поличения предсказаний, загружаем в модель полученный список кодов пользоватлей 
userids = test_list
ids, scores = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)

In [57]:
# а вот тут мы делаем хитро в первую колонку записываем строковые имена пользователей, а в список рекомендаций полученные предсказания
test_prediction = pd.DataFrame({'cookie_id' : test_list_str, 'top_recommendations' : list(ids)})

In [92]:
test_prediction

,cookie_id,top_recommendations
0,00252da2760d4231b366d79eaf6a54a1,"[253946, 117532, 253817, 122614, 209448]"
1,007090a8c8354b42bf538b93785303d3,"[101396, 164246, 165274, 106729, 253678]"
2,00c15cb6e3c84464a0e538e46ced6c46,"[260154, 171332, 253678, 217683, 154356]"
3,00e61f4a8a9f4b8c9fe2913bca7df8d2,"[145455, 185354, 152160, 224389, 131755]"
4,00fa030c79ec4fe5a7b806e6ba60d5ae,"[260154, 117525, 101396, 185354, 190471]"
...,...,...
1220,ff2a45ffbc83417ea01633b48c644f23,"[159332, 187740, 157476, 252102, 182870]"
1221,ff4ffd399268462db31c5ef22277c2bb,"[256132, 171332, 154356, 154248, 101462]"
1222,ff7861c402e84f9fa78ff00f49eb83a1,"[123363, 240927, 105974, 257474, 171543]"
1223,ffa618b6b94444239aee6356fc67af48,"[209422, 209448, 101136, 122614, 185355]"


In [58]:
# вот эта наша функция получения метрики
def precision_n(predictions, df_test_list_top, k=5):
  #recommendations num:
    n_rec_k = len(df_test_list_top)*k
  #true reccomendations:
    matches=0
    for row in range(len(predictions)):
      row_match = set(predictions['top_recommendations'][row]).intersection(set(df_test_list_top['vac'][row]))  
      matches += len(row_match)
  #Results:
    precisions = matches / n_rec_k if n_rec_k != 0 else 0
    return precisions

In [94]:
set(test['vacancy_id_'].iloc[1])
type(test['vacancy_id_'])

pandas.core.frame.DataFrame

In [59]:
df_test_1 = test_prediction
df_test_1['cook'] = test['cookie_id'].values[:,0]
df_test_1['vac'] = test['vacancy_id_'].values[:,0]

In [ ]:
type(df_test_1['vac'])

pandas.core.series.Series

In [64]:
df_test_1

,cookie_id,top_recommendations,cook,vac
0,00252da2760d4231b366d79eaf6a54a1,"[182870, 117532, 253946, 253817, 209448]",00252da2760d4231b366d79eaf6a54a1,"[139502, 133937, 153196, 166055, 168696]"
1,007090a8c8354b42bf538b93785303d3,"[101396, 164246, 165274, 106729, 257747]",007090a8c8354b42bf538b93785303d3,"[101396, 118250, 122614, 152485, 164067]"
2,00c15cb6e3c84464a0e538e46ced6c46,"[260154, 171332, 253678, 101462, 217683]",00c15cb6e3c84464a0e538e46ced6c46,"[139462, 101462, 108276, 108536, 111783]"
3,00e61f4a8a9f4b8c9fe2913bca7df8d2,"[152160, 224389, 185354, 145455, 114560]",00e61f4a8a9f4b8c9fe2913bca7df8d2,"[105265, 109573, 109891, 123183, 182957]"
4,00fa030c79ec4fe5a7b806e6ba60d5ae,"[260154, 101396, 117525, 185354, 245356]",00fa030c79ec4fe5a7b806e6ba60d5ae,"[167159, 211100, 127800, 179478, 189398]"
...,...,...,...,...
1220,ff2a45ffbc83417ea01633b48c644f23,"[198114, 159332, 187740, 157476, 252102]",ff2a45ffbc83417ea01633b48c644f23,"[103615, 112537, 113449, 115446, 119825]"
1221,ff4ffd399268462db31c5ef22277c2bb,"[256132, 171332, 154356, 154248, 101462]",ff4ffd399268462db31c5ef22277c2bb,"[106033, 110884, 119161, 119227, 123306]"
1222,ff7861c402e84f9fa78ff00f49eb83a1,"[123363, 240927, 257474, 171543, 105974]",ff7861c402e84f9fa78ff00f49eb83a1,"[248949, 254085, 108510, 128900, 182528]"
1223,ffa618b6b94444239aee6356fc67af48,"[209422, 209448, 101136, 185355, 122614]",ffa618b6b94444239aee6356fc67af48,"[101136, 117182, 117525, 185355, 223343]"


In [60]:
#type(df_test_1['2'][0])
df_test_1['vac'] = df_test_1['vac'].apply(lambda x: ast.literal_eval(x))


In [98]:
 set(test_prediction['top_recommendations'][2]).intersection(set(df_test_1['vac'][2]))

set()

In [ ]:
test_prediction

,cookie_id,top_recommendations,cook,vac
0,00252da2760d4231b366d79eaf6a54a1,"[122614, 209448, 253946, 117532, 209422]",00252da2760d4231b366d79eaf6a54a1,"[139502, 133937, 153196, 166055, 168696]"
1,0052234b5356480f9dff207888135548,"[230322, 230326, 127887, 182879, 133932]",0052234b5356480f9dff207888135548,"[108861, 134982, 141785, 146245, 153937]"
2,007090a8c8354b42bf538b93785303d3,"[101396, 250327, 164246, 165274, 106729]",007090a8c8354b42bf538b93785303d3,"[101396, 118250, 122614, 152485, 164067]"
3,00c15cb6e3c84464a0e538e46ced6c46,"[171332, 253678, 217683, 101462, 154248]",00c15cb6e3c84464a0e538e46ced6c46,"[139462, 101462, 108276, 108536, 111783]"
4,00d6baa0391e41628da60e2b4a37e19f,"[260154, 202608, 257631, 110421, 112506]",00d6baa0391e41628da60e2b4a37e19f,"[117958, 157903, 159843, 164325, 164331]"
...,...,...,...,...
1785,ff6aeeddb38847c59979efce29225e87,"[101396, 106729, 201312, 257747, 207169]",ff6aeeddb38847c59979efce29225e87,"[112920, 126057, 126404, 127524, 130124]"
1786,ff7861c402e84f9fa78ff00f49eb83a1,"[123363, 197638, 240927, 106228, 241501]",ff7861c402e84f9fa78ff00f49eb83a1,"[248949, 254085, 108510, 128900, 182528]"
1787,ff81d0fe535247ac9df2468f8e57a9e9,"[113088, 150516, 258955, 256041, 103209]",ff81d0fe535247ac9df2468f8e57a9e9,"[107949, 104587, 134588, 136000, 143178]"
1788,ffa618b6b94444239aee6356fc67af48,"[209422, 101136, 209448, 185355, 219770]",ffa618b6b94444239aee6356fc67af48,"[101136, 117182, 117525, 185355, 223343]"


In [ ]:
df_test_1

,cookie_id,top_recommendations,cook,vac
0,00252da2760d4231b366d79eaf6a54a1,"[122614, 209448, 253946, 117532, 209422]",00252da2760d4231b366d79eaf6a54a1,"[139502, 133937, 153196, 166055, 168696]"
1,0052234b5356480f9dff207888135548,"[230322, 230326, 127887, 182879, 133932]",0052234b5356480f9dff207888135548,"[108861, 134982, 141785, 146245, 153937]"
2,007090a8c8354b42bf538b93785303d3,"[101396, 250327, 164246, 165274, 106729]",007090a8c8354b42bf538b93785303d3,"[101396, 118250, 122614, 152485, 164067]"
3,00c15cb6e3c84464a0e538e46ced6c46,"[171332, 253678, 217683, 101462, 154248]",00c15cb6e3c84464a0e538e46ced6c46,"[139462, 101462, 108276, 108536, 111783]"
4,00d6baa0391e41628da60e2b4a37e19f,"[260154, 202608, 257631, 110421, 112506]",00d6baa0391e41628da60e2b4a37e19f,"[117958, 157903, 159843, 164325, 164331]"
...,...,...,...,...
1785,ff6aeeddb38847c59979efce29225e87,"[101396, 106729, 201312, 257747, 207169]",ff6aeeddb38847c59979efce29225e87,"[112920, 126057, 126404, 127524, 130124]"
1786,ff7861c402e84f9fa78ff00f49eb83a1,"[123363, 197638, 240927, 106228, 241501]",ff7861c402e84f9fa78ff00f49eb83a1,"[248949, 254085, 108510, 128900, 182528]"
1787,ff81d0fe535247ac9df2468f8e57a9e9,"[113088, 150516, 258955, 256041, 103209]",ff81d0fe535247ac9df2468f8e57a9e9,"[107949, 104587, 134588, 136000, 143178]"
1788,ffa618b6b94444239aee6356fc67af48,"[209422, 101136, 209448, 185355, 219770]",ffa618b6b94444239aee6356fc67af48,"[101136, 117182, 117525, 185355, 223343]"


In [63]:
# Ну и собственно вот результат
precision_n(test_prediction, df_test_1)

0.05697959183673469